In [2]:
import numpy as np
import gmms

# Example: 2023 Pazarcik Earthquake

## Earthquake parameters

In [5]:
M      = 7.8
hyp_Z  = 10
region = 'Global' # regionalized models for 'Japan', 'Italy', and 'China'

## Multi-segment fault model

In [7]:
ULC_lat = [37.1198,37.5917,37.3949,37.0460,36.1448,37.8854]
ULC_lon = [36.9253,37.3195,36.8728,36.6052,36.1902,37.8136]
flength = [63,54,45,45,108,90]
fwidth  = [21,21,21,21,21,21]
fstrike = [33.5,53,61,32,20,70]
fdip    = [90,90,90,90,90,90]
frake   = [11,11,11,11,11,11]
fZtor   = [0.3,0.3,0.3,0.6,0.9,0.6]

## Site parameters

In [9]:
site_lat  = [36.77006,37.4339]
site_lon  = [35.79005,35.82021]
site_Vs30 = [439,501]

In [10]:
site_Vs30 = np.array(site_Vs30)

## Compute fault parameters

In [12]:
[URC_lat,URC_lon]                         = gmms.faulttools.get_fault_URC(ULC_lat,ULC_lon,flength,fstrike)
[gmm_frake,gmm_fwidth,gmm_fdip,gmm_fZtor] = gmms.faulttools.get_representative_fault(frake,fwidth,flength,fdip,fZtor)
[fnm,frv,fmech]                           = gmms.faulttools.get_faulting_style(gmm_frake)

## Compute site parameters

In [14]:
site_Z1p0 = np.exp((-7.15/4)*np.log((site_Vs30**4+571**4)/(1360**4+571**4))) # BSSA14
site_Z2p5 = np.exp(7.089-1.144*np.log(site_Vs30))                            # CB13

In [15]:
[R_JB,R_rup,R_x] = gmms.distancetools.get_distances(site_lat,site_lon,ULC_lat,ULC_lon,URC_lat,URC_lon,fwidth,flength,fdip,fstrike,fZtor)

In [16]:
for i in range(len(R_JB)):
    print('Site {}:'.format(i+1))
    print('Rjb  = {:.2f} km'.format(R_JB[i]))
    print('Rrup = {:.2f} km'.format(R_rup[i]))
    print('Rx   = {:.2f} km\n'.format(R_x[i]))

Site 1:
Rjb  = 57.35 km
Rrup = 57.36 km
Rx   = -57.22 km

Site 2:
Rjb  = 81.17 km
Rrup = 81.18 km
Rx   = -79.57 km



## Compute ground motion intensity measures

### CAVgm

In [19]:
[mu,phi,tau] = gmms.CampbellBozorgnia2010.CampbellBozorgnia2010(M,gmm_fdip,gmm_fZtor,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv)

In [20]:
for i in range(len(mu)):
    print('Site {}:'.format(i+1))
    print('CAVgm = {:.2f} g-s\n'.format(np.exp(mu[i])))

Site 1:
CAVgm = 0.62 g-s

Site 2:
CAVgm = 0.46 g-s



### CAVdp

In [22]:
flag_CAVgm = 1 # '0' for recorded, '1' for gmm-based
flag_PSV   = 1 # '0' for no PSV check, '1' otherwise

In [23]:
[mu,phi,tau] = gmms.CampbellBozorgnia2011.CampbellBozorgnia2011(M,gmm_fdip,gmm_fZtor,R_JB,R_rup,R_x,site_Vs30,site_Z1p0,site_Z2p5,fnm,frv,flag_CAVgm,flag_PSV)

In [24]:
for i in range(len(mu)):
    print('Site {}:'.format(i+1))
    print('CAVdp = {:.2f} g-s\n'.format(np.exp(mu[i])))

Site 1:
CAVdp = 0.45 g-s

Site 2:
CAVdp = 0.30 g-s



### CAV

In [26]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019(1,M,gmm_fwidth,gmm_fdip,gmm_fZtor,hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)

In [27]:
for i in range(len(mu)):
    print('Site {}:'.format(i+1))
    print('CAV = {:.2f} m/s\n'.format(np.exp(mu[i])))

Site 1:
CAV = 7.81 m/s

Site 2:
CAV = 5.92 m/s



### Ia

In [29]:
[mu,phi,tau] = gmms.CampbellBozorgnia2019.CampbellBozorgnia2019(0,M,gmm_fwidth,gmm_fdip,gmm_fZtor,hyp_Z,R_JB,R_rup,R_x,site_Vs30,site_Z2p5,fnm,frv,region)

In [30]:
for i in range(len(mu)):
    print('Site {}:'.format(i+1))
    print('Ia = {:.2f} m/s\n'.format(np.exp(mu[i])))

Site 1:
Ia = 0.27 m/s

Site 2:
Ia = 0.16 m/s

